# L3: Projections


<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>


In [6]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [7]:
import custom_utils 

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>utils</code> files:</b> To access <code>requirements.txt</code> for this notebook, 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

## Data Loading

In [10]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("MongoDB/airbnb_embeddings", streaming=True, split="train")
dataset = dataset.take(100)
# Convert the dataset to a pandas dataframe
dataset_df = pd.DataFrame(dataset)
dataset_df.head(100)

,_id,listing_url,name,summary,space,description,neighborhood_overview,notes,transit,access,...,images,host,address,availability,review_scores,reviews,weekly_price,monthly_price,text_embeddings,image_embeddings
0,10006546,https://www.airbnb.com/rooms/10006546,Ribeira Charming Duplex,Fantastic duplex apartment with three bedrooms...,Privileged views of the Douro River and Ribeir...,Fantastic duplex apartment with three bedrooms...,"In the neighborhood of the river, you can find...",Lose yourself in the narrow streets and stairc...,Transport: • Metro station and S. Bento railwa...,We are always available to help guests. The ho...,...,"{'thumbnail_url': '', 'medium_url': '', 'pictu...","{'host_id': '51399391', 'host_url': 'https://w...","{'street': 'Porto, Porto, Portugal', 'suburb':...","{'availability_30': 28, 'availability_60': 47,...","{'review_scores_accuracy': 9, 'review_scores_c...","[{'_id': '58663741', 'date': 2016-01-03 05:00:...",NaN,NaN,"[0.0123710884, -0.0180913936, -0.016843712, -0...","[-0.1302358955, 0.1534578055, 0.0199299306, -0..."
1,10021707,https://www.airbnb.com/rooms/10021707,Private Room in Bushwick,Here exists a very cozy room for rent in a sha...,,Here exists a very cozy room for rent in a sha...,,,,,...,"{'thumbnail_url': '', 'medium_url': '', 'pictu...","{'host_id': '11275734', 'host_url': 'https://w...","{'street': 'Brooklyn, NY, United States', 'sub...","{'availability_30': 0, 'availability_60': 0, '...","{'review_scores_accuracy': 10, 'review_scores_...","[{'_id': '61050713', 'date': 2016-01-31 05:00:...",NaN,NaN,"[0.0153845912, -0.0348115042, -0.0093448907, 0...","[0.0340401195, 0.1742489338, -0.1572628617, 0...."
2,1001265,https://www.airbnb.com/rooms/1001265,Ocean View Waikiki Marina w/prkg,A short distance from Honolulu's billion dolla...,Great studio located on Ala Moana across the s...,A short distance from Honolulu's billion dolla...,You can breath ocean as well as aloha.,,Honolulu does have a very good air conditioned...,"Pool, hot tub and tennis",...,"{'thumbnail_url': '', 'medium_url': '', 'pictu...","{'host_id': '5448114', 'host_url': 'https://ww...","{'street': 'Honolulu, HI, United States', 'sub...","{'availability_30': 16, 'availability_60': 46,...","{'review_scores_accuracy': 9, 'review_scores_c...","[{'_id': '4765259', 'date': 2013-05-24 04:00:0...",650.0,2150.0,"[-0.0400562622, -0.0405789167, 0.000644172, 0....","[-0.1640156209, 0.1256971657, 0.6594450474, -0..."
3,10009999,https://www.airbnb.com/rooms/10009999,Horto flat with small garden,One bedroom + sofa-bed in quiet and bucolic ne...,Lovely one bedroom + sofa-bed in the living ro...,One bedroom + sofa-bed in quiet and bucolic ne...,This charming ground floor flat is located in ...,"There´s a table in the living room now, that d...","Easy access to transport (bus, taxi, car) and ...",,...,"{'thumbnail_url': '', 'medium_url': '', 'pictu...","{'host_id': '1282196', 'host_url': 'https://ww...","{'street': 'Rio de Janeiro, Rio de Janeiro, Br...","{'availability_30': 0, 'availability_60': 0, '...","{'review_scores_accuracy': None, 'review_score...",[],1492.0,4849.0,"[-0.063234821, 0.0017937823, -0.0243996996, -0...","[-0.1292964518, 0.037789464, 0.2443587631, 0.0..."
4,10047964,https://www.airbnb.com/rooms/10047964,Charming Flat in Downtown Moda,Fully furnished 3+1 flat decorated with vintag...,The apartment is composed of 1 big bedroom wit...,Fully furnished 3+1 flat decorated with vintag...,With its diversity Moda- Kadikoy is one of the...,,,,...,"{'thumbnail_url': '', 'medium_url': '', 'pictu...","{'host_id': '1241644', 'host_url': 'https://ww...","{'street': 'Kadıköy, İstanbul, Turkey', 'subur...","{'availability_30': 27, 'availability_60': 57,...","{'review_scores_accuracy': 10, 'review_scores_...","[{'_id': '68162172', 'date': 2016-04-02 04:00:...",NaN,NaN,"[0.023723349, 0.0064210771, -0.0339970738, -0....","[-0.1006749049, 0.4022984803, -0.1821258366, 0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [11]:
print("Columns:", dataset_df.columns)

Columns: Index(['_id', 'listing_url', 'name', 'summary', 'space', 'description',
       'neighborhood_overview', 'notes', 'transit', 'access', 'interaction',
       'house_rules', 'property_type', 'room_type', 'bed_type',
       'minimum_nights', 'maximum_nights', 'cancellation_policy',
       'last_scraped', 'calendar_last_scraped', 'first_review', 'last_review',
       'accommodates', 'bedrooms', 'beds', 'number_of_reviews', 'bathrooms',
       'amenities', 'price', 'security_deposit', 'cleaning_fee',
       'extra_people', 'guests_included', 'images', 'host', 'address',
       'availability', 'review_scores', 'reviews', 'weekly_price',
       'monthly_price', 'text_embeddings', 'image_embeddings'],
      dtype='object')


## Document Modelling

In [12]:
listings = custom_utils.process_records(dataset_df)

## Database Creation and Connection

In [13]:
db, collection = custom_utils.connect_to_database()

Connection to MongoDB successful


In [14]:
# Delete any existing records in the collection
collection.delete_many({})

DeleteResult({'n': 0, 'electionId': ObjectId('7fffffff0000000000000002'), 'opTime': {'ts': Timestamp(1723557634, 1), 't': 2}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1723557634, 1), 'signature': {'hash': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 'keyId': 0}}, 'operationTime': Timestamp(1723557634, 1)}, acknowledged=True)

## Data Ingestion

In [15]:
collection.insert_many(listings)
print("Data ingestion into MongoDB completed")

Data ingestion into MongoDB completed


## Vector Search Index defintion

In [16]:
# Create vector search index
custom_utils.setup_vector_search_index_with_filter(collection=collection)

Creating index...
New index created successfully: vector_index_with_filter


<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note:</b> If the output of the previous cell is <code>Error creating vector search index: Duplicate Index</code> you may proceed to the next cell if you intend to still use a previously created index.</p>

## Handling User Query

In [17]:
from pydantic import BaseModel
from typing import Optional

# Note: Ensure that the  projection document in the projection stage matches the search result model.
class SearchResultItem(BaseModel):
    name: str
    accommodates: Optional[int] = None
    address: custom_utils.Address
    summary: Optional[str] = None
    space: Optional[str] = None
    neighborhood_overview: Optional[str] = None
    notes: Optional[str] = None
    score: Optional[float]=None

In [18]:
from IPython.display import display, HTML

def handle_user_query(query, db, collection, stages=[], vector_index="vector_index_text"):
    get_knowledge = custom_utils.vector_search_with_filter(query, db, collection, stages, vector_index)

    if not get_knowledge:
        return "No results found.", "No source information available."
    
    print("List of all fields of the first document, before model conformance")
    print(get_knowledge[0].keys())

    search_results_models = [
        SearchResultItem(**result)
        for result in get_knowledge
    ]

    search_results_df = pd.DataFrame([item.dict() for item in search_results_models])

    completion = custom_utils.openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system", 
                "content": "You are a airbnb listing recommendation system."},
            {
                "role": "user", 
                "content": f"Answer this user query: {query} with the following context:\n{search_results_df}"
            }
        ]
    )
    system_response = completion.choices[0].message.content
    print(f"- User Question:\n{query}\n")
    print(f"- System Response:\n{system_response}\n")
    display(HTML(search_results_df.to_html()))
    return system_response

## Adding A Projection Stage

In [19]:

projection_stage = {
    "$project": {
        "_id": 0,  
        "name": 1,
        "accommodates": 1,
        "address.street": 1,
        "address.government_area": 1, 
        "address.market": 1,
        "address.country": 1, 
        "address.country_code": 1, 
        "address.location.type": 1, 
        "address.location.coordinates": 1,  
        "address.location.is_location_exact": 1,
        "summary": 1,
        "space": 1,  
        "neighborhood_overview": 1, 
        "notes": 1, 
        "score": {"$meta": "vectorSearchScore"} 
    }
}

additional_stages = [projection_stage]

## Results

In [23]:
query = """
I need to take my family of 10 to see the Buckingham Palace"
"""
handle_user_query(
    query, 
    db, 
    collection, 
    additional_stages, 
    vector_index="vector_index_with_filter"
)

Total time for the execution to complete on the database server: 0.162001 milliseconds
List of all fields of the first document, before model conformance
dict_keys(['name', 'summary', 'space', 'neighborhood_overview', 'notes', 'accommodates', 'address', 'score'])
- User Question:

I need to take my family of 10 to see the Buckingham Palace"


- System Response:
Based on your query to visit Buckingham Palace with a family of 10, I recommend the following Airbnb listings:

1. **Private OceanFront - Bathtub Beach. Spacious House**  
   - Accommodates: 14 guests
   - Address: Laie, HI, United States
   - Summary: Ocean Living! Secluded Secret Beach! Less than...
   - Space: Private & Open Space! Secluded Secret Beach! T...
   - Neighborhood Overview: The beach is one of the best kept secrets on O...
   - Notes: 
   - Score: 0.622261

2. **Bondi Beach Dreaming 3-Bed House**  
   - Accommodates: 8 guests
   - Address: Bondi Beach, NSW, Australia
   - Summary: This peaceful house in North Bon

,name,accommodates,address,summary,space,neighborhood_overview,notes,score
0,Homely Room in 5-Star New Condo@MTR,2,"{'street': 'Mongkok, Kowloon, Hong Kong', 'government_area': 'Yau Tsim Mong', 'market': 'Hong Kong', 'country': 'Hong Kong', 'country_code': 'HK', 'location': {'type': 'Point', 'coordinates': [114.17094, 22.32074], 'is_location_exact': False}}","Located in Mongkok, close to everything. 2min walk to both Mongkok and Mongkok East station. Gym, sauna and swimming pool (in summer) are available in the clubhouse. You'll have a private double room. Washroom and kitchen are shared with host. We are family of 3, my husband, 1y old son and me. The guest bedroom can accommodate two people, the 3rd person has to sleep on the couch (3'x6') in the living room.","You will stay with my son, my husband and me. We couple love travelling very much and have been to more than 35 countries in the past few years. We like to share our travel tips and photos with everyone. There is a luxury clubhouse in my building, with gym and swimming pool. The building is newly built and it's the most luxury one in Mongkok area.",Many restaurants and shops nearby.,"Just feel as home. We will give you all assistance. The 3rd guest is allowed to sleep on the sofa in the living room, and it's subject to an extra charge HK$250 per night.",0.664003
1,"Big, Bright & Convenient Sheung Wan",2,"{'street': 'Hong Kong, Hong Kong Island, Hong Kong', 'government_area': 'Central & Western', 'market': 'Hong Kong', 'country': 'Hong Kong', 'country_code': 'HK', 'location': {'type': 'Point', 'coordinates': [114.15007, 22.28422], 'is_location_exact': True}}",Hollywood Rd Studio Only 1 apartment per floor 650sqft! Fast Wifi and Kitchen! Daily Housekeeping! NEAR 3min walk from MTR 3min walk from Escalators 5min walk from LKF (Please note I live here so some belongings kept in wardrobe),Huge Luxury Studio Serviced Apartment 650sqm! That means daily house keeping and all Amendities!,LKF Escalators IFC Mall SOHO food Wellington St Food!,"I actually live in the studio, so my belongings will be in a wardrobe space",0.638806
2,UWS Brownstone Near Central Park,3,"{'street': 'New York, NY, United States', 'government_area': 'Upper West Side', 'market': 'New York', 'country': 'United States', 'country_code': 'US', 'location': {'type': 'Point', 'coordinates': [-73.9696, 40.78558], 'is_location_exact': True}}","This spacious one bedroom apartment has a bonus space with two extra beds as well as a fold out sofa. Close to the park, the subway, museums and restaurants. This apartment is the perfect quiet getaway in the middle of Manhattan.","This charming one-bedroom apartment in a landmark brownstone townhouse in the Upper West Side area of Manhattan has high-ceilings, a fireplace and two planted, south-facing balconies overlooking a beautiful garden. (The balcony off of the living room is about 5"" X 8"" and has a small table and two chairs where two people can have a nice meal. The balcony off of the bedroom is about 5"" X 14' and there is a chaise longue and a sitting chair, along with two side tables. Both have outdoor lights, overlook a nice garden, and have plantings.) There is a queen-sized bed in the private bedroom with closing door. There is also a very high quality full sized sofa bed in the living room. This beautiful home is on an historic tree-lined block, between Central Park West and Columbus Avenue, near major museums, shopping, and subways. The owner prefers to accommodate two adults. Because of the circumstances, only guests who are comfortable being reasonably quiet are encouraged to stay. Thi",,,0.635115
3,The LES Apartment,3,"{'street': 'New York, NY, United States', 'government_area': 'Lower East Side', 'market': 'New York', 'country': 'United States', 'country_code': 'US', 'location': {'type': 'Point', 'coordinates': [-73.98944, 40.72063], 'is_location_exact': False}}","Dear guests, Would you just like to live in the heart of the city that you would be eligib

'Based on your query to visit Buckingham Palace with a family of 10, I recommend the following Airbnb listings:\n\n1. **Private OceanFront - Bathtub Beach. Spacious House**  \n   - Accommodates: 14 guests\n   - Address: Laie, HI, United States\n   - Summary: Ocean Living! Secluded Secret Beach! Less than...\n   - Space: Private & Open Space! Secluded Secret Beach! T...\n   - Neighborhood Overview: The beach is one of the best kept secrets on O...\n   - Notes: \n   - Score: 0.622261\n\n2. **Bondi Beach Dreaming 3-Bed House**  \n   - Accommodates: 8 guests\n   - Address: Bondi Beach, NSW, Australia\n   - Summary: This peaceful house in North Bondi is 300m to ...\n   - Space: Serene space with three bedrooms, including a ...\n   - Neighborhood Overview: North Bondi is laid back with great food and c...\n   - Notes: Please nominate the correct number of guests b...\n   - Score: 0.615411\n\n3. **Greenwich Fun and Luxury**  \n   - Accommodates: 6 guests\n   - Address: Greenwich, NSW, Austral